<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Visualising-some-data-from-the-LOPEX'93-dataset" data-toc-modified-id="Visualising-some-data-from-the-LOPEX'93-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Visualising some data from the LOPEX'93 dataset</a></span><ul class="toc-item"><li><span><a href="#Comments" data-toc-modified-id="Comments-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Comments</a></span></li></ul></li><li><span><a href="#Model-inversion" data-toc-modified-id="Model-inversion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model inversion</a></span><ul class="toc-item"><li><span><a href="#Simple-PROSPECT-inversion" data-toc-modified-id="Simple-PROSPECT-inversion-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Simple PROSPECT inversion</a></span><ul class="toc-item"><li><span><a href="#Some-exercises..." data-toc-modified-id="Some-exercises...-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Some exercises...</a></span></li></ul></li></ul></li><li><span><a href="#Final-remarks" data-toc-modified-id="Final-remarks-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Final remarks</a></span></li></ul></div>

<div style="float:right">
    <table>
    <tr>
        <td> <img src="../figs/pangeos-small-1.png" alt="PANGEOS" style="width:200px;height:45px;"/> 
        <td> <img src="../figs/kcl_logo.png" alt="King's College London" style="width:54px;height:40px;"/> 
        <td> <img src="../figs/nceo_logo.png" alt="NCEO" style="width:200px;height:40px;"/> 
        <td> <img src="../figs/multiply_logo.png" alt="H2020 Multiply" style="width:40px;height:40px;"/>
    </tr>
    </table>
</div>
&nbsp;

# Bayesian inversion of the PROSPECT model. The role of uncertainty

**Author:** Jose Gómez-Dans (NCEO & UCL)  `jose.gomez-dans@kcl.ac.uk`



## Introduction

The previous notebook explored using a simple function fitting approach to retrieve biophysical parameters from leaf reflectance and transmittance spectra. In this notebook, we'll extend the previous work by considering the effect of **uncertainty** in the retrieval. This is a crucial aspect of any retrieval, as it allows us to quantify the confidence we have in the retrieved parameters.

Earlier, we only used a simple concept of uncertainty: we did a set of random starts for the minimisation, and looked at how the results changed. This is only a part of the uncertainty in the retrieval. We can probably think of the following sources of uncertainty:
* **Measurement uncertainty**: this is the uncertainty in the measurements themselves.
* **Model uncertainty**: this is the uncertainty in the model itself. In our case, we used the PROSPECT model, which is a simple model that is known to have some limitations. 
* **Retrieval uncertainty**: this is the uncertainty in the retrieval process itself. In the previous notebook, this is the uncertainty that arises from starting on different points in the parameter space, but for other methods it might be to do with limitations in the inversion algorithm.
* **Representation uncertainty** In many cases, we use models that describe whole leaf and/or canopy dynamics, and only sample a small part of the variance in the real world.
* **Prior uncertainty**: all retrieval methods constrain the parameter space in some way or another. You can think of this as *prior information*. In the previous notebook, we used a uniform prior, but in many cases, we might have more information about the parameters we are trying to retrieve. How we encode that information will affect the retrieval.


We will first start by looking at the LOPEX'93 data again...

<div class="alert alert-block alert-warning">
    This notebook has a number of commented out cells. These are exercises for you to try out. You can uncomment them by removing the `#` at the beginning of the line.
</div>




In [ ]:
%load_ext autoreload
%autoreload 2
from pangeos_uq.prosail_funcs import (
    read_lopex_sample,
    the_cost_function_covariance,
    optimise_random_starts,
    calculate_mean_and_covariance,
)

from pangeos_uq.mcmc import generate_samples
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import ipywidgets as widgets

## Visualising uncertainty from the LOPEX'93 dataset

The [LOPEX'93 dataset](https://data.ecosis.org/dataset/13aef0ce-dd6f-4b35-91d9-28932e506c41/resource/4029b5d3-2b84-46e3-8fd8-c801d86cf6f1/download/leaf-optical-properties-experiment-93-lopex93.pdf) contains five replicates per sample. We will use this to estimate the uncertainty in the measurements. 

We will assume that the measurements are corrupted by additive zero mean Gaussian noise. For reflectance, this can be written as:
$$
\vec{\rho}_{\text{meas}} = \vec{\rho}_{\text{true}} + \vec{\epsilon}(0, \mathcal{C}_{obs}),
$$

where $\vec{\rho}_{\text{meas}}$ is the observed reflectance for all wavelengths, $\vec{\rho}_{\text{true}}$ is the true reflectance, and $\vec{\epsilon}(0, \mathcal{C}_{obs})$ is a zero mean Gaussian noise with covariance $\mathcal{C}_{obs}$. A similar expression holds for transmittance.

In the next couple of cells, you'll select some samples from the LOPEX'93 database, and we'll use them to calculate the uncertainty in the measurements. We'll visualise the combined reflectance and transmittance **correlation matrix**. This is just a scaled version of the covariance matrix, and it tells us how the different bands and measurements are correlated.

In [ ]:
widgets.interact(
    read_lopex_sample, sample_no=widgets.IntSlider(min=1, max=116, value=23)
)
refl, trans = read_lopex_sample.data

In [ ]:
mean_rho, mean_tau, cov_matrix, inv_cov_matrix, correlation_matrix = (
    calculate_mean_and_covariance(refl, trans)
)

plt.imshow(
    correlation_matrix,
    interpolation="nearest",
    cmap="inferno",
    vmin=-1,
    vmax=1,
)
plt.axvline(2101, color="g", lw=2)
plt.axhline(2101, color="g", lw=2)
plt.colorbar(shrink=0.5);

The correlation matrix can be interpted using the green horizontal and vertical lines that divide into 4 quadrants. The top left quadrant is the correlation between the reflectance bands, the bottom right quadrant is the correlation between the transmittance bands, and the other two quadrants are the correlation between reflectance and transmittance bands. The main diagonal is just 1s, as it's the correlation of a band with itself. But we can see some very clear patterns in the correlation matrix.

Can you try to explain the patterns you see in the correlation matrix?

## Model inversion

We will now invert the model using a normal log-likelihood function. The log-likelihood function is given by:

$$
J(\mathbf{x}) = \frac{1}{2} \left( M(\mathbf{x}) - \vec{y} \right)^{\top} \mathcal{C}^{-1} \left( M(\mathbf{x}) - \vec{y} \right).
$$

In essence, we'll be trying to see the effect of the uncertainty in the retrievals. For the retrievals, we'll use a time-consuming MCMC approach, that has the benefit of being simple to follow and to implement. The MCMC approach will allow us to sample the posterior distribution of the parameters, and to quantify the uncertainty in the retrieval.

We'll run three experiments:
1. A simple inversion with fixed uncertainty and no spectral correlation (e.g. the covariance matrix is diagonal with a fixed value).
2. As above, but with the diagonal now reflecting the uncertainty in the measurements.
3. As above, but with the full covariance matrix reflecting the uncertainty in the measurements, including spectral correlation.

We will run this with reflectance, but you're welcome to extend this to transmittance too.

We will define a few common variables that will be used in the inversion. You can change things around a bit, but the main parameter to change is the `n_iterations` variable, which controls the number of samples in the MCMC chain. The more samples, the better the estimate of the posterior distribution, but the longer the code will take to run. Values of less than 1000 are probably not very useful, and values of more than 10000 probably take too long to run.

Since we're mostly interested in looking at the effect of uncertainty, we can probably just start by using our gradient descent method to find the maximum likelihood estimate of the parameters. This will give us a good starting point for the MCMC chain, and ensure that we focus on where the posterior distribution is likely to be.

For reference, on my laptop, 5000 samples take about 1 minute to run.

<div class="alert alert-block alert-warning">
    Some of the code here will take time to run slowly if you set `n_iterations` to a large number. You can test how long this will take on your computer by running the code with a small number of iterations first (say 1000) and extending it to a more sensible value once you know how long it'll take.
</div>

In [ ]:
lobound = np.array([0.8, 0, 0, 0, 0.0043, 0.0017])
hibound = np.array([2.8, 80, 20, 1, 0.0439, 0.0152])

# Get a starting point for the MCMC close to where the action happens

df, _, _ = optimise_random_starts(
    mean_rho,
    None,
    n_tries=5,
    lobound=lobound,
    hibound=hibound,
    verbose=False,
    do_plots=False,
)
# Extract the average solution values. Discard last column as that's cost
initial_value = df.iloc[-2, :-1].values
# Need to get a reasonable proposal step... divide std dev by 10 to
# get to the right scale(ish)
transitions = df.iloc[-1:, :-1].values / 10.0

n_iterations = 10_000

In [ ]:
# this_inv_cov_matrix = np.eye(4202) * (np.diagonal(inv_cov_matrix).mean())
# cost_simple = []


# def logpdf(x: np.ndarray) -> float:
#     """Log likelihood function for the MCMC sampler. This function takes up
#     some variables from the main namespace, so I don't have to boether about
#     adding them as arguments to the function. This is a bit of a hack, but
#     there we go"""
#     lobound = np.array([0.8, 0, 0, 0, 0.0043, 0.0017])
#     hibound = np.ndarray = np.array([2.8, 80, 20, 1, 0.0439, 0.0152])

#     if np.any(x < lobound) or np.any(x > hibound):
#         # Proposed value is outside the bounds, never accept
#         return -np.inf
#     # Uncomment below to get different measurements into the inversion
#     # Using only reflectance measurements
#     cost = the_cost_function_covariance(x, mean_rho, None, this_inv_cov_matrix)
#     # Using **both** reflectance and transmittance measurements
#     # cost = the_cost_function_covariance(x, mean_rho,
#     #               mean_trans, this_inv_cov_matrix)
#     # Using **only** transmittance measurements
#     # cost = the_cost_function_covariance(x, mean_rho,
#     #               None, this_inv_cov_matrix)
#     cost_simple.append(cost)
#     return cost


# vals_simple = np.array(
#     list(
#         generate_samples(
#             initial_value, n_iterations, logpdf, scaling=transitions
#         )
#     )
# )

In [ ]:
# this_inv_cov_matrix = np.eye(4202) * inv_cov_matrix.diagonal()

# cost_diag = []


# def logpdf(x: np.ndarray) -> float:
#     """Log likelihood function for the MCMC sampler. This function takes up
#     some variables from the main namespace, so I don't have to boether about
#     adding them as arguments to the function. This is a bit of a hack, but
#     there we go"""
#     lobound = np.array([0.8, 0, 0, 0, 0.0043, 0.0017])
#     hibound = np.ndarray = np.array([2.8, 80, 20, 1, 0.0439, 0.0152])

#     if np.any(x < lobound) or np.any(x > hibound):
#         # Proposed value is outside the bounds, never accept
#         return -np.inf
#     # Uncomment below to get different measurements into the inversion
#     # Using only reflectance measurements
#     cost = the_cost_function_covariance(x, mean_rho, None, this_inv_cov_matrix)
#     # Using **both** reflectance and transmittance measurements
#     # cost = the_cost_function_covariance(x, mean_rho,
#     #               mean_trans, this_inv_cov_matrix)
#     # Using **only** transmittance measurements
#     # cost = the_cost_function_covariance(x, mean_rho,
#     #               None, this_inv_cov_matrix)

#     cost_diag.append(cost)
#     return cost


# vals_diag = np.array(
#     list(
#         generate_samples(
#             initial_value, n_iterations, logpdf, scaling=transitions
#         )
#     )
# )

In [ ]:
# this_inv_cov_matrix = inv_cov_matrix

# cost_full = []


# def logpdf(x: np.ndarray) -> float:
#     """Log likelihood function for the MCMC sampler. This function takes up
#     some variables from the main namespace, so I don't have to boether about
#     adding them as arguments to the function. This is a bit of a hack, but
#     there we go"""
#     lobound = np.array([0.8, 0, 0, 0, 0.0043, 0.0017])
#     hibound = np.ndarray = np.array([2.8, 80, 20, 1, 0.0439, 0.0152])

#     if np.any(x < lobound) or np.any(x > hibound):
#         # Proposed value is outside the bounds, never accept
#         return -np.inf
#     # Uncomment below to get different measurements into the inversion
#     # Using only reflectance measurements
#     cost = the_cost_function_covariance(x, mean_rho, None, this_inv_cov_matrix)
#     # Using **both** reflectance and transmittance measurements
#     # cost = the_cost_function_covariance(x, mean_rho,
#     #               mean_trans, this_inv_cov_matrix)
#     # Using **only** transmittance measurements
#     # cost = the_cost_function_covariance(x, mean_rho,
#     #               None, this_inv_cov_matrix)
#     cost_full.append(cost)
#     return cost


# vals_full = np.array(
#     list(
#         generate_samples(
#             initial_value, n_iterations, logpdf, scaling=transitions
#         )
#     )
# )

We have now run the MCMC sampler. In a real life MCMC sampling problem, we would have to run the sampler for a long time to ensure convergence. We would also have to check for convergence, and to ensure that the samples are independent. We would also have to check that the samples are representative of the posterior distribution.

Since this isn't real life, we'll just take the last 1000 samples from the chain and assume they're representative of the posterior distribution. We'll then plot the marginal posterior distributions of the parameters, and the joint posterior distributions of the parameters.

You can save your sampling results below. We also provide a sample file with the results of the sampling for a large number of iterations. You can use this to plot the results without having to run the MCMC sampler, or to compare things.

In [ ]:
# # Save your inversion results to an npz file
# np.savez(
#     "prospect_mcmc_example_YOURNAME.npz",
#     vals_simple=vals_simple,
#     vals_diag=vals_diag,
#     vals_full=vals_full,
#     cost_simple=cost_simple,
#     cost_diag=cost_diag,
#     cost_full=cost_full,
# )

In [ ]:
# # Retrieve the reference MCMC samples from disk
f = np.load("prospect_mcmc_example_NEW.npz")
vals_simple_reference = f["vals_simple"][-5000:, :]
vals_diag_reference = f["vals_diag"][-5000:, :]
vals_full_reference = f["vals_full"][-5000:, :]
cost_simple_reference = f["cost_simple"]
cost_diag_reference = f["cost_diag"]
cost_full_reference = f["cost_full"]

# vals_simple_reference = vals_simple
# vals_diag_reference = vals_diag
# vals_full_reference = vals_full

In [ ]:
# Packs the reference samples into pandas DataFrames for easy plotting

df_simple = pd.DataFrame(
    vals_simple_reference, columns=["n", "cab", "car", "cbrown", "cw", "cm"]
)
df_diag = pd.DataFrame(
    vals_diag_reference, columns=["n", "cab", "car", "cbrown", "cw", "cm"]
)

df_full = pd.DataFrame(
    vals_full_reference, columns=["n", "cab", "car", "cbrown", "cw", "cm"]
)

df_simple["source"] = "Simple"
df_diag["source"] = "Diagonal"
df_full["source"] = "Full"

df_combined = pd.concat([df_simple, df_diag, df_full])

In [ ]:
# Create a pairplot of the MCMC samples. Take every 25th sample to speed up
# plotting and to deal with serial correlation in the MCMC samples.

#sns.pairplot(df_combined.iloc[::25, :], hue="source")


In [ ]:
# # Do a simple parameter spread plot

# # # Create a figure with 2x3 subplots
# fig, axes = plt.subplots(3, 2, figsize=(8, 8))

# # Flatten axes array for easier indexing
# axes = axes.flatten()

# # Plot violin plots for each variable
# for i, var in enumerate(df_simple.columns[:-1]):
#     #sns.stripplot(x='source', y=var, data=df_combined, ax=axes[i])
#     sns.violinplot(
#         x="source",
#         y=var,
#         data=df_combined,
#         ax=axes[i],
#         hue="source"
# )
#     axes[i].set_title(var)
# fig.tight_layout()


The plots take a considerable amount of time to run, so you might want to skip this step, and plot some previously prepared samples. Here's a pair plot of the samples, and a plot of the posterior distribution of the parameters.
![Pair plot](../figs/mcmc_pairplot.png)


![Sample distribution](../figs/mcmc_prospect.png)


### Try this plots with your data!

You can run the sampler above with your data, and plot the results. There are some commented out bits and pieces to give you hints on how to do this. Some suggestions:
1. Run with reflectance, and then with transmittance, and finally, reflectance and transmittance together.
2. Play around with the different inverse covariance matrices
3. Try different priors for the parameters. Currently, it's just a bounded space (uniform "prior"), but you could use a Gaussian prior, or a mixture of Gaussians, or a Student's t distribution, etc.

For example, assume that you wanted to have a Gaussian prior with a mean vector $\mu$, and prior covariances $\sigma_{cab}, \dots$. You would then have to modify the log-likelihood function to include the prior. Note that it is often a good idea to still keep the boundaries on the parameters, so as not to run prospect with meaningless parameters (e.g. $N < 0$).

```python
# assume you have a prior mean and covariance vectors
mu = np.array([2, 40, 20, 0.5, 0.01, 0.01]) # for example
# The covariance matrix is a diagonal matrix with the prior variances 
# given by whatever you want. Here I double the mean value and square it.
cov = np.eye(6) * (mu*2)**2 # for example
prior = scipy.stats.multivariate_normal(mu, cov)
# Define this inside `logpdf`, and then add it to the log-likelihood:
return cost + prob.logpdf(x)
```


<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.